<a href="https://colab.research.google.com/github/MostafaMahmoudegy10/Cart/blob/main/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Importing** the Dependeciees

In [2]:
import numpy as np
import pandas as pd
# searching for a text in the document
import re  # regular expression
# natural language tool kit
# stopwords => الى هو الكلام الى ملوش لازمه مش هيضيف حاجه للفيتشر فيكتور
from nltk.corpus import stopwords
# takes a word and remove the prefix and suffix of it
from nltk.stem.porter import PorterStemmer
# convert text into feature vector
from sklearn.feature_extraction.text import TfidfVectorizer
# split our data set into training data and test data
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
# accuracy scope
from sklearn.metrics import accuracy_score

In [3]:
import nltk
nltk.download('stopwords')



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [7]:
# printing the stopwords
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

# **Data Collection and Data Pre Processing**

In [93]:
fake  = pd.read_csv('/content/Fake.csv', on_bad_lines='warn');
true = pd.read_csv('/content/True.csv',on_bad_lines='warn');

/tmp/ipython-input-1333172392.py:1: ParserWarning: Skipping line 13775: expected 4 fields, saw 21
Skipping line 14170: expected 4 fields, saw 9
Skipping line 24951: expected 4 fields, saw 49
Skipping line 25816: expected 4 fields, saw 20
Skipping line 31254: expected 4 fields, saw 6
Skipping line 31770: expected 4 fields, saw 9
Skipping line 34985: expected 4 fields, saw 24

  fake  = pd.read_csv('/content/Fake.csv', on_bad_lines='warn');
/tmp/ipython-input-1333172392.py:2: ParserWarning: Skipping line 1236: expected 4 fields, saw 7
Skipping line 1659: expected 4 fields, saw 50
Skipping line 11343: expected 4 fields, saw 16
Skipping line 13055: expected 4 fields, saw 12
Skipping line 13518: expected 4 fields, saw 50
Skipping line 26051: expected 4 fields, saw 16
Skipping line 26440: expected 4 fields, saw 38
Skipping line 29941: expected 4 fields, saw 12

  true = pd.read_csv('/content/True.csv',on_bad_lines='warn');


In [22]:
# add column label to determine if the data is fake or true
fake['label'] = 0
true['label'] = 1
data=pd.concat([fake,true],axis=0)
news_dataset = data.sample(frac=1).reset_index(drop=True)


In [106]:
news_dataset.head()

,title,text,subject,date,label,content
0,WHY UNEDUCATED SOMALI REFUGEES Who Don’t Speak...,"A few weeks ago, we reported on the first fema...",left-news,"Apr 29, 2017",0,uneduc somali refuge speak english flee arizon...
1,Trump Got Caught Doing Something REALLY CREEP...,Donald Trump s most recent secretive actions t...,News,"February 11, 2017",0,trump got caught someth realli creepi vacat fl...
2,JUST IN: “Pit Bull” Attorney For Special Couns...,Is there a single person left on Robert Muelle...,left-news,"Dec 8, 2017",0,pit bull attorney special counsel robert muell...
3,Survey: Top Ten Fears of 2015,Truthstream MediaOut of 88 potential horrors ...,Middle-east,"February 29, 2016",0,survey top ten fear truthstream mediaout poten...
4,DISASTER CAPITALISTS: How Bill And Hillary’s “...,Hillary was Secretary of State an unimaginable...,left-news,"Oct 7, 2016",0,disast capitalist bill hillari clinton foundat...


In [23]:
news_dataset.shape

(82643, 5)

In [27]:
news_dataset.head(5)

,title,text,subject,date,label
0,WHY UNEDUCATED SOMALI REFUGEES Who Don’t Speak...,"A few weeks ago, we reported on the first fema...",left-news,"Apr 29, 2017",0
1,Trump Got Caught Doing Something REALLY CREEP...,Donald Trump s most recent secretive actions t...,News,"February 11, 2017",0
2,JUST IN: “Pit Bull” Attorney For Special Couns...,Is there a single person left on Robert Muelle...,left-news,"Dec 8, 2017",0
3,Survey: Top Ten Fears of 2015,Truthstream MediaOut of 88 potential horrors ...,Middle-east,"February 29, 2016",0
4,DISASTER CAPITALISTS: How Bill And Hillary’s “...,Hillary was Secretary of State an unimaginable...,left-news,"Oct 7, 2016",0


In [30]:
news_dataset.isnull().sum()

,0
title,0
text,0
subject,0
date,0
label,0


In [29]:
#replacing the missing values with null
news_dataset=news_dataset.fillna('mkya')

In [94]:
# merging the subject column and title column for prediction
news_dataset['content']=news_dataset['title']+' '+news_dataset['text']

In [107]:
# make sure it is right
news_dataset['content']

,content
0,uneduc somali refuge speak english flee arizon...
1,trump got caught someth realli creepi vacat fl...
2,pit bull attorney special counsel robert muell...
3,survey top ten fear truthstream mediaout poten...
4,disast capitalist bill hillari clinton foundat...
...,...
82638,viewer demand shepard smith fire fox news left...
82639,bitter brzezinski ap report go low snarkfest p...
82640,republican benghazi witch hunt went flame than...
82641,shock summari dnc convent far leav anyth crazi...


# Stemming process :
   process of reducing a word to its root
   acting , actor, actress => act

In [39]:
port_stem=PorterStemmer()

In [96]:
def stemming(content):
  stemmed_content=re.sub('[^a-zA-Z]',' ',content)
  stemmed_content=stemmed_content.lower()
  stemmed_content=stemmed_content.split()
  stop_words=stopwords.words('english')
  stemmed_content=[ port_stem.stem(word) for word in stemmed_content if not word in stop_words ]
  stemmed_content=' '.join(stemmed_content)
  return stemmed_content

In [105]:
# apply the stemming into our prediction column
news_dataset['content']=news_dataset['content'].apply(stemming)

KeyboardInterrupt: 

# Split To Train and Test Data

In [98]:
x = news_dataset['content'].values
y = news_dataset['label'].values

X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.2, stratify=y, random_state=2)

In [99]:
# convert data into meangifull numbers "vectors"
vectorizer=TfidfVectorizer()

X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec  = vectorizer.transform(X_test)

# Trainning Our Model :Logistic Regression Model

In [100]:
model=LogisticRegression(max_iter=1000)

In [101]:
model.fit(X_train_vec,Y_train)

LogisticRegression(max_iter=1000)

# Evaluation

**Accuracy Scope**

In [102]:
X_train_prediction = model.predict(X_train_vec)
training_accuracy = accuracy_score(Y_train, X_train_prediction)

X_test_prediction = model.predict(X_test_vec)
testing_accuracy = accuracy_score(Y_test, X_test_prediction)

In [103]:
print("Training Accuracy :", training_accuracy)
print("Testing Accuracy  :", testing_accuracy)

Training Accuracy : 0.994116223492755
Testing Accuracy  : 0.9919535362090871


In [113]:
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

# TF-IDF على كل البيانات
vectorizer = TfidfVectorizer()
X_vec = vectorizer.fit_transform(x)

# الموديل
model = LogisticRegression(max_iter=1000, random_state=2)

# 5-fold cross-validation
cv_scores = cross_val_score(model, X_vec, y, cv=5, scoring='accuracy')

print("Cross-Validation Accuracy Scores:", cv_scores)
print("Mean CV Accuracy:", cv_scores.mean())
print("Std CV Accuracy:", cv_scores.std())

Cross-Validation Accuracy Scores: [0.99080404 0.99225603 0.99231653 0.99092449 0.99171104]
Mean CV Accuracy: 0.9916024276789592
Std CV Accuracy: 0.0006396818729563479
